In [99]:
import re
import pandas as pd
import numpy as np
import nltk
#from nltk.tokenize import TreebankWordTokenize
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [38]:
tweets=pd.read_csv("C:/Users/heera/Desktop/GLabs_DSMP_New-masters/twitter/train.csv",encoding='latin')
tweets

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
...,...,...,...
99984,99996,0,@Cupcake seems like a repeating problem hop...
99985,99997,1,@cupcake__ arrrr we both replied to each other...
99986,99998,0,@CuPcAkE_2120 ya i thought so
99987,99999,1,@Cupcake_Dollie Yes. Yes. I'm glad you had mor...


In [39]:
#total words in corpus
def total_tokens_raw(tweet):
    """
    Find total raw tokens
    
    Args:
    tweet - str - tweet  text
    
    Returns:
    int - count of tokens in tweet
    """
    
    return len(word_tokenize(tweet))

In [40]:
word_tokenize("This is an interesting game")

['This', 'is', 'an', 'interesting', 'game']

In [41]:
tweets['raw_tokens_count']=tweets['SentimentText'].map(total_tokens_raw)

In [42]:
tweets.head()

,ItemID,Sentiment,SentimentText,raw_tokens_count
0,1,0,is so sad for my APL frie...,12
1,2,0,I missed the New Moon trail...,7
2,3,1,omg its already 7:30 :O,6
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...,31
4,5,0,i think mi bf is cheating on me!!! ...,12


In [64]:
tweets.iloc[0][2]

'                     is so sad for my APL friend.............'

In [65]:
#total tokens in corpus
print("total tokenns".format(sum(tweets['raw_tokens_count'])))

total tokenns


In [66]:
#total unique words in corpus

#1)Comvine all tweets
combined_tweets=' '.join(tweets['SentimentText'])

#2)word tokenixation on tweets
tokenized_tweets= word_tokenize(combined_tweets)

#3)Create set
set_tokenized_tweets=set(tokenized_tweets)

#4)Find length of set
print('Total unique tweets are - {}'.format(len(set_tokenized_tweets)))

Total unique tweets are - 133560


In [67]:
combined_tweets[:150]

'                     is so sad for my APL friend.............                    I missed the New Moon trailer...               omg its already 7:30 :'

In [70]:
#Total unique words in corpus after lowercase
#1)Comvine all tweets
combined_tweets=' '.join(tweets['SentimentText'])
lower_combined_tweets=combined_tweets.lower()

#2)word tokenixation on tweets
tokenized_tweets_lower= word_tokenize(lower_combined_tweets)

#3)Create set
set_tokenized_tweets_lower=set(tokenized_tweets_lower)

#4)Find length of set
print('Total unique tweets are - {}'.format(len(set_tokenized_tweets_lower)))

Total unique tweets are - 117940


In [71]:
#Total unique words in corpus after lowercase, stemming / lemmatization

def lower_stem_tweet(tweet):
    """
    Lower cases and applies stemming
    
    Args:
    tweet - str - tweet text
    
    Returns:
    int - count of tokens in tweet
    """
    return len(word_tokenize(tweet))

In [75]:
combined_lc_st_tweets=' '.join([lower_stem_tweet(tweet) for tweet in tweets['SentimentText']])
print('Total unique tweets are - {}'.format(len(combined_lc_st_tweets)))

TypeError: sequence item 0: expected str instance, int found

In [76]:
# Load english stop words
stops=(stopwords.words('english'))

NameError: name 'stopwords' is not defined

In [77]:
#Preprocessing steps

#1)Hashtags
#2)Mentions @
#3)Tagging
#4)Links
#5)Numbers
#6)Punctuation
#7)smileys/Emojis
#8)Timestamps
#9)HTML
#10)Dtaes
#11)Images(<img alt=)


In [78]:
def normalize_tweet(tweet):
    #lowercase
    tweet=tweet.lower()
    #Substitute mentions
    tweet=re.sub('@\w+', '__mention__',tweet)
    #Substitute hashtags
    tweet=re.sub('#\w+', '__hashtag__',tweet)
    return tweet

In [81]:
tweets['norm tweet']=tweets['SentimentText'].map(normalize_tweet)


In [85]:
tweets[tweets['SentimentText'].str.contains('#')]

,ItemID,Sentiment,SentimentText,raw_tokens_count,norm tweet
131,132,0,#3turnoffwords this shit sucks,5,__hashtag__ this shit sucks
132,133,0,#asylm J2 panel is over. Guess it's back to n...,15,__hashtag__ j2 panel is over. guess it's back...
133,134,1,#poemsunder140 ....started by @shannonelyse1,7,__hashtag__ ....started by __mention__
134,135,0,#squarespace brighten my bad day! i never win...,12,__hashtag__ brighten my bad day! i never win ...
135,136,0,"#Susan Boyle didnt win! mh well, diversity wa...",18,"__hashtag__ boyle didnt win! mh well, diversi..."
...,...,...,...,...,...
99967,99979,1,@ctcash @buildingateam @diabetescure @chocolat...,23,__mention__ __mention__ __mention__ __mention_...
99971,99983,0,@CTerry1985 That's the thing; the new raft of ...,22,__mention__ that's the thing; the new raft of ...
99973,99985,1,@ctham #FollowFriday,4,__mention__ __hashtag__
99974,99986,0,@ctham #awaresg You are not wrong. But from a ...,33,__mention__ __hashtag__ you are not wrong. but...


In [ ]:
### Sentiment analysis

#1)import countvectorizer
#2)create X and Y
#3)Create train and test dataset
#4)Fit and transform vectors x_train and just transform x_tst
#5)Fit model
#6)predict
#7)Print CLF report



In [89]:
cv= CountVectorizer()

In [92]:
X=tweets['SentimentText']
y=tweets['Sentiment']


#Split train test

X_train,X_test,y_train,y_test=train_test_split(X, y, train_size=0.7, random_state=1)

C:\Users\heera\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2178: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [93]:
X_train_vec=cv.fit_transform(X_train)
X_test_vec=cv.transform(X_test)

In [101]:
rf=RandomForestClassifier()
lr=LogisticRegression()
vot_hard=VotingClassifier([('rf',rf),('lr',lr)],voting='hard')
vot_soft=VotingClassifier([('rf',rf),('lr',lr)],voting='soft')
# Fit and predict
print('Random Forest Results')
print('---------------------')
rf.fit(X_train_vec, y_train)
y_pred = rf.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')
print('Logistic Regression Results')
print('---------------------')
lr.fit(X_train_vec, y_train)
y_pred = lr.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')
print('Voting Classifier Hard Results')
print('---------------------')
vot_hard.fit(X_train_vec, y_train)
y_pred = vot_hard.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')
print('Voting Classifier Soft Results')
print('---------------------')
vot_soft.fit(X_train_vec, y_train)
y_pred = vot_soft.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')

NameError: name 'LogisticRegression' is not defined

In [100]:
# Create X & y
X = tweets['norm_tweet']
y = tweets['Sentiment']
# Split data in to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)
# Fit and transform vectors for X_train
X_train_vec = cv.fit_transform(X_train)
# Transform X_test to vectors
X_test_vec = cv.transform(X_test)
# Fit and predict
print('Random Forest Results')
print('---------------------')
rf.fit(X_train_vec, y_train)
y_pred = rf.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')
print('Logistic Regression Results')
print('---------------------')
lr.fit(X_train_vec, y_train)
y_pred = lr.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')
print('Voting Classifier Hard Results')
print('---------------------')
vot_hard.fit(X_train_vec, y_train)
y_pred = vot_hard.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')
print('Voting Classifier Soft Results')
print('---------------------')
vot_soft.fit(X_train_vec, y_train)
y_pred = vot_soft.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')

KeyError: 'norm_tweet'

In [103]:
cv=CountVectorizer(ngram_range=(1,3))


# Create X & y
X = tweets['norm_tweet']
y = tweets['Sentiment']
# Split data in to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)
# Fit and transform vectors for X_train
X_train_vec = cv.fit_transform(X_train)
# Transform X_test to vectors
X_test_vec = cv.transform(X_test)
# Fit and predict

print('Random Forest Results')
print('---------------------')
rf.fit(X_train_vec, y_train)
y_pred = rf.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')
print('Logistic Regression Results')
print('---------------------')
lr.fit(X_train_vec, y_train)
y_pred = lr.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')
print('Voting Classifier Hard Results')
print('---------------------')
vot_hard.fit(X_train_vec, y_train)
y_pred = vot_hard.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')
print('Voting Classifier Soft Results')
print('---------------------')
vot_soft.fit(X_train_vec, y_train)
y_pred = vot_soft.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print('')

KeyError: 'norm_tweet'